In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from timeit import timeit
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from joblib import dump, load
from imblearn.under_sampling import RandomUnderSampler

In [2]:
from compiledataset_reveng import load_dataset, compile_dataset

PATH = "/home/hampus/miun/master_thesis/Datasets"

datasets = {}

# dataset: pd.DataFrame = load_dataset(PATH + "/ORNL", "data_a.csv")
# datasets["ROAD"] = dataset.to_dict("records")

dataset: pd.DataFrame = load_dataset(PATH + "/Survival", "data.csv")
datasets["Survival"] = dataset.to_dict("records")

# dataset: pd.DataFrame = load_dataset(PATH + "/Hisingen", "data.csv")
# datasets["Hisingen"] = dataset.to_dict("records")


df = compile_dataset(datasets)
df.drop(columns=["d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "DLC", "t"], inplace=True, errors="ignore")

dataset = None # Release memory, as it isn't used for now
datasets = None

display(df)

,ID,data,data_hex,name,class,dataset,type,Label
0,672,0100000000000000011011100001110111001100000001...,40006e1dcc04e300,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
1,809,0000110010110000011111110001010000010001001000...,0cb07f1411200014,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
2,688,1111111101111111000000000000010100011100000000...,ff7f00051c000000,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
3,880,0000000000100000000000000000000000000000000000...,0020000000000000,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
4,1087,0001000001000000011000001111111101011010011110...,104060ff5a790800,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
...,...,...,...,...,...,...,...,...
117168,304,0000011110000000000000001111111100011100100000...,078000ff1c800cc5,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
117169,305,0001001010000000000000000000000011100000011111...,12800000e07f0c2d,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
117170,320,0000000000000000000000000000000000001100001000...,000000000c202c6e,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0
117171,688,1111111101111111000000000000010101011000000000...,ff7f000558000000,FreeDrivingData_20180323_SONATA,Sonata,Survival,none,0


In [6]:
display(df.loc[df["ID"] == 704, "data"])

7         0001010000000000000000000000000000000000000000...
29        0001010000000000000000000000000000000000000000...
47        0001010000000000000000000000000000000000000000...
67        0001010000000000000000000000000000000000000000...
86        0001010000000000000000000000000000000000000000...
                                ...                        
117079    0001010000000000000000000000000000000000000000...
117100    0001010000000000000000000000000000000000000000...
117117    0001010000000000000000000000000000000000000000...
117138    0001010000000000000000000000000000000000000000...
117158    0001010000000000000000000000000000000000000000...
Name: data, Length: 6006, dtype: object

In [11]:
id_704 = df.loc[df["ID"] == 704, "data"]

l_ids = []

def get_is(l):
    l_id = []
    for s in l:
        for i, c in enumerate(s):
            if c == "1" and i not in l_id:
                l_id.append(i)
    l_ids.append(l_id)

df.groupby("ID")["data"].apply(lambda x: get_is(x))


display(l_ids)


[[44, 47, 52, 53, 56, 46, 55, 57, 60, 63, 54, 58, 61, 62, 59, 45],
 [1,
  2,
  6,
  9,
  10,
  11,
  12,
  13,
  16,
  18,
  20,
  21,
  28,
  31,
  42,
  54,
  19,
  7,
  17,
  3,
  4,
  5],
 [1, 2, 3, 4, 5, 6, 7, 8, 14, 15, 34, 37, 38, 0, 39],
 [5,
  7,
  8,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  34,
  36,
  38,
  40,
  52,
  53,
  57,
  59,
  62,
  6,
  39,
  55,
  58,
  61,
  35,
  54,
  63,
  37,
  56,
  4,
  60,
  32,
  33,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  3,
  0,
  1,
  2,
  9,
  10,
  11,
  12,
  13,
  14,
  15],
 [5,
  7,
  8,
  32,
  34,
  35,
  36,
  37,
  39,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  52,
  53,
  56,
  57,
  58,
  61,
  62,
  63,
  38,
  55,
  59,
  54,
  60,
  4,
  6,
  0,
  1,
  2,
  3,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  33],
 [35,
  36,
  38,
  50,
  52,
  53,
  61,
  62,
  42,
  55,
  57,
  58,
  59,
  60,
  37,
  54,
  63,
  44,
  47,
  45,
  56,
  46,
  43,
  34],
 [10, 15, 19, 24, 25, 26, 27, 28, 29, 30, 31, 40, 41, 42,